In [54]:
import tvm

import numpy as np

import tvm.te as te
import tvm.relax as rx
import tvm.tir as tir

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)


from tvm.relax.frontend import nn

In [55]:
testSuites = {}


def register(func):
    if func.__name__ in testSuites:
        raise Exception("Duplicated test suite name: " + func.__name__)
    else:
        testSuites[func.__name__] = func
    return func

In [56]:
@register
def test_Parameter():
    param0 = nn.Parameter((4, 4), "float32")
    print(f"param0: {param0}")
    print(f"param0.data: {param0.data}")
    param0.data = np.random.randn(4, 4).astype("float32")
    print(f"param0.data: {param0.data}")

    param1 = nn.Parameter((4, 4), "float32")
    print(f"param1.dtype: {param1.dtype}")
    # change the dtype of param1
    param1.to("int32")
    print(f"param1.dtype: {param1.dtype}")


testSuites["test_Parameter"]()

param0: Tensor([4, 4], "float32")
param0.data: None
param0.data: [[-0.05631397  2.2376904  -0.29770148 -0.65897965]
 [ 0.13761097 -0.9416021   1.3386246   0.70951796]
 [ 1.8505553  -0.17791286 -0.7759552   0.00947368]
 [-0.23217033 -1.293148   -0.29679146  1.0891947 ]]
param1.dtype: float32
param1.dtype: int32


In [57]:
@register
def test_Module():
    # define a custom model
    class MyModel(nn.Module):
        def __init__(self):
            super(MyModel, self).__init__()
            self.fc1 = nn.Linear(784, 256)
            self.relu1 = nn.ReLU()
            self.fc2 = nn.Linear(256, 10)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu1(x)
            x = self.fc2(x)
            return x

    # NOTE use the `export_tvm` to export the model to IRModule
    # `spec` is the dictionary where
    # * the keys are the method names to export
    # * the values are dictionaries mapping parameter names to their specifications.
    mod, param_spec = MyModel().export_tvm(
        spec={"forward": {"x": nn.spec.Tensor((1, 784), "float32")}}
    )
    showmod(mod)
    print(param_spec)
    # [('fc1.weight', Tensor([256, 784], "float32")), ('fc1.bias', Tensor([256], "float32")),
    # ('fc2.weight', Tensor([10, 256], "float32")), ('fc2.bias', Tensor([10], "float32"))]

    mod = rx.get_pipeline("zero")(mod)
    target = tvm.target.Target("llvm", host="llvm")
    ex = rx.build(mod, target=target)
    vm = rx.VirtualMachine(ex, tvm.cpu())

    _data = np.random.randn(1, 784).astype("float32")
    tvm_data = tvm.nd.array(_data, device=tvm.cpu())

    # generate the random values for the weights and biases
    params = [np.random.rand(*param.shape).astype("float32") for _, param in param_spec]
    params = [tvm.nd.array(param, device=tvm.cpu()) for param in params]
    print(vm["forward"](tvm_data, *params).numpy())


testSuites["test_Module"]()

[('fc1.weight', Tensor([256, 784], "float32")), ('fc1.bias', Tensor([256], "float32")), ('fc2.weight', Tensor([10, 256], "float32")), ('fc2.bias', Tensor([10], "float32"))]


[[828.9319  836.31006 842.11    820.2735  824.8049  833.589   899.34827
  764.0125  840.08215 798.19147]]


In [58]:
@register
def test_module_list():
    class MyModule(nn.Module):
        def __init__(self):
            self.layers = nn.ModuleList([nn.Linear(4, 4, bias=False), nn.ReLU()])

        def forward(self, x: nn.Tensor):
            return self.layers(x)

    mod = MyModule()
    print(dict(mod.named_parameters()))
    print(mod.state_dict())


testSuites["test_module_list"]()

{'layers.0.weight': Tensor([4, 4], "float32")}
OrderedDict([('layers.0.weight', Tensor([4, 4], "float32"))])


In [59]:
@register
def test_wrap_nested():
    pass
    # pv0 = rx.Tuple([1, 2, 3])
    # BUG Don't know how to use wrap_nested
    # print(wrap_nested(pv0, "tensor0"))


testSuites["test_wrap_nested"]()